In [2]:
import pandas as pd
import re

In [3]:

bank_statement_df = pd.read_excel(r"D:\NPS_phase1\BANK STMT PHASE 4\Bank_Nepal.xlsx", skiprows=1)


In [4]:
bank_statement_df.head()

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number
0,1,24208406906844000001,-35260.0,D,IMEREMIT#772930111,225544/GLBBNPKA,NPS-IF-3130638/GLBBNPKA,2023-08-28T12:58:00,NaN
1,2,24208406906844000001,-151184.0,D,IMEREMIT#9496608477,225545/GLBBNPKA,NPS-IF-3130639/GLBBNPKA,2023-08-28T12:58:00,NaN
2,3,24208406906844000001,5100.0,C,9813264371,"NPS1,18717698,{Remarks}",100003984626,2023-08-28T12:57:00,NaN
3,4,24208406906844000001,1800.0,C,9845408504,IMEPAY:9869023444,"18717582,pay",2023-08-28T12:54:00,NaN
4,5,24208406906844000001,-4500.0,D,Txn:C7562206B2574 G,225542/100003984601,C7562206B2574/100003984601,2023-08-28T12:54:00,NaN


In [7]:
for index, row in bank_statement_df.iterrows():
    # print(bank_statement_df.iloc[index]['Transaction ID'])
    for roww in (row['Desc2'] , row['Desc3']):
        id = ''
        if "NPS-IF-" in roww:
            id = re.findall(r'[0-9]{7}',roww)
            bank_statement_df.loc[index, 'Transaction Id'] = id[0]
            break
        
        
        elif "10000" in roww:
            id = re.findall(r'10*[0-9]{7}',roww)
            bank_statement_df.loc[index, 'Transaction Id'] = id[0].replace("10000","")
            break
        
        elif "FTMS-" in roww:
            id = re.findall(r'[0-9]{6}',roww)
            bank_statement_df.loc[index, 'Transaction Id'] = id[0]
            
        elif re.findall(r'\d{7}-.*', roww):
            id = re.findall(r'[0-9]{7}', roww)
            bank_statement_df.loc[index, 'Transaction ID'] = id[0]

In [8]:
bank_statement_df.head()

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number,Transaction ID
0,1,24208406906844000001,-35260.0,D,IMEREMIT#772930111,225544/GLBBNPKA,NPS-IF-3130638/GLBBNPKA,2023-08-28T12:58:00,NaN,3130638
1,2,24208406906844000001,-151184.0,D,IMEREMIT#9496608477,225545/GLBBNPKA,NPS-IF-3130639/GLBBNPKA,2023-08-28T12:58:00,NaN,3130639
2,3,24208406906844000001,5100.0,C,9813264371,"NPS1,18717698,{Remarks}",100003984626,2023-08-28T12:57:00,NaN,3984626
3,4,24208406906844000001,1800.0,C,9845408504,IMEPAY:9869023444,"18717582,pay",2023-08-28T12:54:00,NaN,NaN
4,5,24208406906844000001,-4500.0,D,Txn:C7562206B2574 G,225542/100003984601,C7562206B2574/100003984601,2023-08-28T12:54:00,NaN,3984601


In [9]:
bank_statement_df['Transaction ID'].count()

389

In [10]:
bank_statement_df['Date'] = pd.to_datetime(bank_statement_df['Date'], format="%Y-%m-%dT%H:%M:%S").dt.date

In [12]:
def standard_format_phase_4(row):
        """
        Determine transaction mode and amount from bank statement data.

        Parameters:
            row (Series): A row of bank statement data.

        Returns:
            Series: Mode (CR/DR) and amount.
        """
        if row['Txn Type'] == 'D' :
            mode = 'DR'
            amount = abs(row['Amount'])
        elif row['Txn Type'] == 'C':
            mode = 'CR'
            amount = row['Amount']
        else:
            mode = None
            amount = None
        return pd.Series([mode, amount], index=['Mode', 'Amount'])


In [13]:
bank_statement_df[['Mode', 'Amount']] = bank_statement_df.apply(standard_format_phase_4, axis=1)


In [14]:
bank_statement_df

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number,Transaction ID,Mode
0,1,24208406906844000001,35260.0,D,IMEREMIT#772930111,225544/GLBBNPKA,NPS-IF-3130638/GLBBNPKA,2023-08-28,NaN,3130638,DR
1,2,24208406906844000001,151184.0,D,IMEREMIT#9496608477,225545/GLBBNPKA,NPS-IF-3130639/GLBBNPKA,2023-08-28,NaN,3130639,DR
2,3,24208406906844000001,5100.0,C,9813264371,"NPS1,18717698,{Remarks}",100003984626,2023-08-28,NaN,3984626,CR
3,4,24208406906844000001,1800.0,C,9845408504,IMEPAY:9869023444,"18717582,pay",2023-08-28,NaN,NaN,CR
4,5,24208406906844000001,4500.0,D,Txn:C7562206B2574 G,225542/100003984601,C7562206B2574/100003984601,2023-08-28,NaN,3984601,DR
...,...,...,...,...,...,...,...,...,...,...,...
543,544,24208406906844000001,10.0,C,9807870200,IMEPAY:9824853622,"18718080,hhh",2023-08-28,NaN,NaN,CR
544,545,24208406906844000001,850.0,C,9868016254,IMEPAY:9812414874,"18718079,kbb",2023-08-28,NaN,NaN,CR
545,546,24208406906844000001,15000.0,D,NEPALREMIT#Fund Tra,225551/SIDDNPKA,NPS-IF-3130679/SIDDNPKA,2023-08-28,NaN,3130679,DR
546,547,24208406906844000001,500000.0,D,mOmIA-NPS4i2JGA5,225550/NEBLNPKA,FTMS-181408/NEBLNPKA,2023-08-28,NaN,181408,DR
